# Imports

In [1]:
from tqdm import tqdm
import joblib
import re
import spacy
import nltk
import pandas as pd

import sys
import json
#import stanza
#stanza.download('pt')
import numpy as np
# install and import amr utils
sys.path.append('../')
sys.path.append("../src/")
from features.nlp_preprocess import *
#!pip install ../amr-utils
from amr_utils.amr_readers import AMR_Reader

In [2]:
# folder = '../POS-tagger-portuguese-nltk/trained_POS_taggers/'
# tagger_nltk = joblib.load(folder+'POS_tagger_brill.pkl')

# Create dict annotation

In [3]:
path_lp = '../data/raw/little_prince.txt'
path_opisums = '../data/raw/opisums.txt'
path_news = '../data/raw/news.txt'
path_sci = '../data/raw/science.txt'

list_paths = [
    ('sci',path_sci),
    
    ('lp', path_lp),
    ('opisums',path_opisums),
    ('news',path_news),
]

In [4]:
df_features = pd.DataFrame({
    'sentence_id': [],
    "corpus_name": [],
    'parent': [],
    'child': [],
    'parent_pos': [],
    'child_pos': [],
    'parent_ner': [],
    'child_ner': [],
    'dependency_role': [],
    'parent_position': [],
    'child_position': [],
    'label':[]
})

nlp_model_spacy = spacy.load("pt_core_news_lg")

list_dict_an = []
for corpus_name, path in list_paths:
    
    print(f"""##################################################
# Running: {corpus_name}
##################################################""")
        
    # usa reader de amr para fazer o parsing
    reader = AMR_Reader()
    amrs = reader.load(path, remove_wiki=True)
    
    # cria bloco de textos anotados
    with open(path, 'r') as file:
        str_corpus = file.read()
    blocos = str_corpus.split('\n\n')
    blocos = [bloco for bloco in blocos if bloco != ""]
        
    list_dicts = []
    # para cada sentenca, cria dict com os nós e vértices
    for i,amr in tqdm(enumerate(amrs), total = len(amrs)):
        
        texto_anotado = blocos[i]
        dict_aligments = parse_alignment(texto_anotado)
        
        amr_string = amr.amr_string() # obtem string formatada amr        
        
        dict_annotation = amr_to_dict(amr) 
        dict_annotation.update({'corpus_name': corpus_name})
        dict_annotation.update({'dict_aligments': dict_aligments})
        list_dicts.append(dict_annotation)
        
    print('Anotando texto ...')    
    for dict_an in tqdm(list_dicts):
        
        tokens = dict_an['tokens_nltk']
        
        if tokens == []:
            tokens = dict_an['tok pt']
        
        # tok_pos = pos_tagger_nltk(
        # tokenized_sentence = tokens,
        # tagger_nltk = tagger_nltk
        # )
          
        dict_spacy = extract_feat_spacy(
            text = dict_an['snt'],
            nlp_model = nlp_model_spacy)
        
                
        dict_an.update(dict_spacy)
        #dict_an.update({'tok pos': tok_pos})
        dict_an.update({"corpus_name": corpus_name})
        
        list_dict_an.append(dict_an)

    print()
    
with open('../data/processed/annotated_text.json', 'w') as f:
    json.dump(list_dict_an, f)

cannot deinvert attribute: ('s2', ':op2-of', 'and')


##################################################
# Running: sci
##################################################
[amr] Loading AMRs from file: ../data/raw/science.txt


100%|██████████| 160/160 [00:00<00:00, 3987.48it/s]


Anotando texto ...


100%|██████████| 160/160 [00:00<00:00, 175.70it/s]


...

100%|██████████| 404/404 [00:02<00:00, 185.23it/s]



##################################################
# Running: news
##################################################
[amr] Loading AMRs from file: ../data/raw/news.txt


100%|██████████| 870/870 [00:00<00:00, 6918.16it/s]


Anotando texto ...


100%|██████████| 870/870 [00:04<00:00, 211.15it/s]
